In [160]:
import scipy
import numpy as np
import pandas as pd

In [156]:
class matlab_reader():
    def __init__(self,file):
        self.file_path = file
        self.data = scipy.io.loadmat(file)
        self.data_info = scipy.io.whosmat(file)

        self.data_airs = self.data['Airs']

        # to get night values use mat_data['Airs']['NH'][x][x][x][x][1][x][x][x]
        # to get day values use   mat_data['Airs']['NH'][x][x][x][x][0][x][x][x]

        # to get tp values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][0]
        # to get bg values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][1]
        # to get A  values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][2]
        # to get ha values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][3]
        # to get k  values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][4]
        # to get l  values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][5]
        # to get m  values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][6]
        # to get mfx values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][7]
        # to get mfy values use mat_data['Airs']['NH'][x][x][x][x][x][x][x][8]


        # new_data[0] gives the temperature matrix 501x501 for altitude at lowest level 24
        # new_data[1] gives the temperature matrix 501x501 for altitude at lowest level 30
        # new_data[2] gives the temperature matrix 501x501 for altitude at lowest level 36
        # new_data[3] gives the temperature matrix 501x501 for altitude at lowest level 42
        # new_data[4] gives the temperature matrix 501x501 for altitude at lowest level 48
        
        self.mappings = {"day" : 0,
                        "night": 1,
                        "tp" : 0,
                        "bg" : 1,
                        "a" : 2,
                        "ha" : 3,
                        "k" : 4,
                        "l" : 5,
                        "m" : 6,
                        "mfx" : 7,
                        "mfy" : 8,
                        24 : 0,
                        30 : 1,
                        36 : 2,
                        42 : 3,
                        48 : 4}
    def select(self,
        hemisphere = 'NH',
        daytime = 'Night',
        data_field = 'tp',
        altitude = None):
        ''' 
        Returns a selection on the dataset filtered based on the input values
        hemisphere : selects what hemisphere to select data, North or South
        daytime : selects what time of day to collect values, Night or Day
        data_field : selects data field from tp (temperature) , bg , a , ha , k, l , m , mfx and mfy.
        '''
        
        data_ = self.data_airs[hemisphere.upper()][0][0][0][0][self.mappings[daytime.lower()]][0][0][self.mappings[data_field.lower()]]
        
        # transpose the data to have 5, 501x501 arrays, each for a different altitude level
        new_data = np.transpose(np.transpose(data_, (2, 0, 1)), (0, 2, 1)) 
        
        if altitude is not None:
            new_data = new_data[self.mappings[altitude]]

        print(np.shape(new_data))
        return new_data

# Example 

matlab = matlab_reader('AIRS_40KM_2022/20220125_AIRS_3DST-1_40km_grid.mat')

matlab.select(
    hemisphere= 'sh',
    data_field='bg',
    daytime='day',
    altitude=24)      


In [167]:
class grid():
    def __init__(self):
        self.nh_lat = np.genfromtxt('Airs_nh_lat.csv', delimiter=',')
        self.nh_lon = np.genfromtxt('Airs_nh_lon.csv', delimiter=',')
        self.sh_lat = np.genfromtxt('Airs_sh_lat.csv', delimiter=',')
        self.sh_lon = np.genfromtxt('Airs_sh_lon.csv', delimiter=',')
    



In [168]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

data = np.random.rand(501, 501)
coords = np.zeros((501, 501, 2))
coords[:, :, 0] = np.linspace(-90, 90, 501)[:, np.newaxis]
coords[:, :, 1] = np.linspace(-180, 180, 501)

# Define the map projection and plot the map
m = Basemap(projection='mill', lat_0=0, lon_0=0)
m.drawcoastlines()
m.drawcountries()

# Define the coordinates of the data
lats = coords[:, :, 0]
lons = coords[:, :, 1]

# Plot the temperature values on the map using a color map
temp = data
x, y = m(lons, lats)  # Convert the coordinates to the map projection
m.pcolormesh(x, y, temp, cmap='coolwarm')  # Plot the temperature values

# Add a color bar to the plot
cbar = plt.colorbar()
cbar.set_label('Temperature')

# Show the plot
plt.show()

Fontconfig warning: ignoring UTF-8: not a valid region tag


ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [204]:

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# First we specify Coordinate Refference System for Map Projection
# We will use Mercator, which is a cylindrical, conformal projection. 
# It has bery large distortion at high latitudes, cannot 
# fully reach the polar regions.
projection = ccrs.Mercator()

# Specify CRS, that will be used to tell the code, where should our data be plotted
crs = ccrs.PlateCarree()

# Now we will create axes object having specific projection 
plt.figure(dpi=150)
ax = plt.axes(projection=projection, frameon=True)

# Draw gridlines in degrees over Mercator map
gl = ax.gridlines(crs=crs, draw_labels=True,
                  linewidth=.6, color='gray', alpha=0.5, linestyle='-.')
gl.xlabel_style = {"size" : 7}
gl.ylabel_style = {"size" : 7}

# To plot borders and coastlines, we can use cartopy feature
import cartopy.feature as cf
ax.add_feature(cf.COASTLINE.with_scale("50m"), lw=0.5)
ax.add_feature(cf.BORDERS.with_scale("50m"), lw=0.3)

# Now, we will specify extent of our map in minimum/maximum longitude/latitude
# Note that these values are specified in degrees of longitude and degrees of latitude
# However, we can specify them in any crs that we want, but we need to provide appropriate
# crs argument in ax.set_extent
lon_min = -20
lon_max = 45
lat_min = 34
lat_max = 60

# crs is PlateCarree -> we are explicitly telling axes, that we are creating bounds that are in degrees
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=crs)
plt.title(f"Temperature anomaly over Europe in {original_data.valid_time.dt.strftime('%B %Y').values}")
plt.show()